# Evaluate RAG

In [1]:
import os
import pandas as pd
import sys
import random
sys.path.append("../")
from src.config import Configuration
from src.robot import RAGRobot
from src.service.provider import ProviderService

config = Configuration()
provider = ProviderService()
robot = RAGRobot(provider=provider)

test_folder = "training_program"
rag = robot.training_prop_rag.chain
# rag
config.enable_tracing(project="Evaluation")

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded index doc
loaded index doc
loaded index doc


In [2]:
path = f"../data/{test_folder}/test"
files = os.listdir(path)
all_files = [f"{path}/{f}" for f in files]
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

df = df[['question', 'answer', 'wrong', 'doc_id']]
df['category'] = test_folder
df.shape
# df.head()
df = df.sample(frac = 1)

# df = df.iloc[:40]

In [3]:
df.shape

(22, 5)

In [4]:
import time

print("Before the sleep statement")
time.sleep(5)
print("After the sleep statement")

Before the sleep statement
After the sleep statement


In [5]:
records = []
for i, row in df.iterrows():
    prompt = ""
    labels = ["A", "B", "C", "D"]
    record = {**row.to_dict()}
    question = row['question']
    if i%3 == 0:
        time.sleep(20)
    try:
        choices = row['wrong'].split("\n- ")[1:]
        resp = rag.invoke({"question": question})
    except:
        print("error")
        time.sleep(30)
        continue
    # update
    record['pred_doc_id'] = resp.document.id
    record['RAG_answer'] = resp.answer
    rand_idx = random.randint(0, len(labels) - 1)

    record['correct_label'] = labels.pop(rand_idx)
    prompt += f"{record['correct_label']}. {record['answer']}\n"
    while len(labels) > 0:
        rand_idx = random.randint(0, len(labels) - 1)
        prompt += f"{labels.pop(rand_idx)}. {choices.pop()}\n"
    record['multi_prompt'] = prompt
    records.append(record)

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


error
error
error
error


In [6]:
len(records)

18

In [7]:
ndf = pd.DataFrame(records)
ndf.to_csv(f"../experiments/results/eval_{test_folder}_gemini_rag.csv")

In [8]:
df.columns

Index(['question', 'answer', 'wrong', 'doc_id', 'category'], dtype='object')

In [10]:
import pandas as pd

df = pd.read_csv("../experiments/results/eval_major_rag.csv")

check_nan = df.isnull().sum()
check_nan

Unnamed: 0        0
question          0
answer            0
wrong             0
doc_id            0
category          0
pred_doc_id       0
RAG_answer       41
correct_label     0
multi_prompt      0
dtype: int64

In [11]:
df.head()

,Unnamed: 0,question,answer,wrong,doc_id,category,pred_doc_id,RAG_answer,correct_label,multi_prompt
0,0,Ngành Xã hội học tại TDTU có những hướng chuyê...,Quản lý xã hội và xã hội truyền thông đại chúng,- Tâm lý học xã hội\n- Nhân học xã hội\n- Xã h...,e4ce2db8-0c1c-4634-a7b4-3bc03310521c,major,dd0573ae-29a2-4afa-b5c1-303be63b8bfa,NaN,D,D. Quản lý xã hội và xã hội truyền thông đại c...
1,1,Ngành Xã hội học tại TDTU có thời gian đào tạo...,4 năm,- 3 năm\n- 5 năm\n- 6 năm\n- 7 năm,e4ce2db8-0c1c-4634-a7b4-3bc03310521c,major,dd0573ae-29a2-4afa-b5c1-303be63b8bfa,NaN,D,D. 4 năm\nC. 7 năm\nB. 6 năm\nA. 5 năm\n
2,2,Ngành Quy hoạch vùng và đô thị đào tạo về nhữn...,Quy hoạch đô thị và nông thôn,- Kiến trúc\n- Xây dựng\n- Kỹ thuật môi trường...,680b5b13-33b0-40b9-9e74-f3f4dd11a04d,major,ded27576-2a47-46d2-a351-cd88df27a109,"Quy hoạch sử dụng đất, quy hoạch đô thị, quy h...",D,D. Quy hoạch đô thị và nông thôn\nB. Quản lý đ...
3,3,Ngành Quy hoạch vùng và đô thị đào tạo trong b...,4 năm hoặc 5 năm,- 2 năm\n- 3 năm\n- 6 năm\n- 7 năm,680b5b13-33b0-40b9-9e74-f3f4dd11a04d,major,index,4 năm,A,A. 4 năm hoặc 5 năm\nD. 7 năm\nC. 6 năm\nB. 3 ...
4,4,Ngành Ngôn ngữ Anh tại Trường Đại học Tôn Đức ...,"Ngữ âm – âm vị, hình thái học, cú pháp học, ng...",- Toán cao cấp\n- Vật lý hạt nhân\n- Hóa học h...,fd6ea7bc-ad3e-4268-8994-dd6b512a30b9,major,dd0573ae-29a2-4afa-b5c1-303be63b8bfa,NaN,C,"C. Ngữ âm – âm vị, hình thái học, cú pháp học,..."


# CHECK 1 FILE

In [9]:
import pandas as pd
import os
from sklearn.metrics import classification_report

path = "../experiments/results"
all_files = ['eval_major_gemini_rag.csv', 'eval_training_program_gemini_rag.csv','eval_uni_gemini_rag.csv']


file = all_files[2]

df = pd.read_csv(os.path.join(path, file))
df = df.fillna("None")

y = df['doc_id']
yhat = df['pred_doc_id']

score = 0
for a,b in zip(y, yhat):
    if a == b:
        score += 1

print(score/len(y))
print(score)
print(len(y))


labels = df['correct_label']
preds = df['RAG_label']
report = classification_report(y_true=labels, y_pred=preds)
print(report)


0.7368421052631579
14
19
              precision    recall  f1-score   support

           A       0.86      1.00      0.92         6
           B       1.00      1.00      1.00         2
           C       1.00      0.60      0.75         5
           D       1.00      0.83      0.91         6
        None       0.00      0.00      0.00         0

    accuracy                           0.84        19
   macro avg       0.77      0.69      0.72        19
weighted avg       0.95      0.84      0.88        19



/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Check

In [1]:
import os
import pandas as pd
import sys
sys.path.append("../")
from src.config import Configuration
from src.service.provider import ProviderService

config = Configuration()
provider = ProviderService()
# robot = RAGRobot(provider=provider)

ai = provider.get_simple_gemini_pro()


path = "../experiments/results"
all_files = ['eval_major_gemini_rag.csv', 'eval_training_program_gemini_rag.csv','eval_uni_gemini_rag.csv']

def choose(ai, pred, choices):
    prompt = f"""Đáp án là "{pred}"
    .Vậy phải lựa chọn đáp án nào trong danh sách dưới đây
    ```
    {choices}
    ```
    """
    a = ai(prompt)
    return a

# for file in all_files:
#     ans = []
#     df = pd.read_csv(os.path.join(path, file))
#     df = df.fillna("None")
#     for i, row in df.iterrows():
#         a = choose(ai, row['RAG_answer'], row['multi_prompt'])
#         if "." in a:
#             a = a.strip()[0]
#         if len(a) == 0 or a == "":
#             a = "None"
#         ans.append(a)
#         print(a)
#     df['RAG_label'] = ans
#     df.to_csv(os.path.join(path, file))

# file = all_files[0]
df = pd.concat((pd.read_csv(os.path.join(path, f)) for f in all_files), ignore_index=True)
# print(df.shape)
# df.columns

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df.head()

,question,answer,wrong,doc_id,category,pred_doc_id,RAG_answer,correct_label,multi_prompt,RAG_label
0,Ngành Khoa học máy tính tại Trường Đại học Tôn...,"Tiêu chuẩn, Chất lượng cao, Đại học bằng tiếng...",- Khoa học dữ liệu\n- Trí tuệ nhân tạo\n- An n...,94ee4dc5-31b3-407a-9553-8ecf72b83c6f,major,index,NaN,A,"A. Tiêu chuẩn, Chất lượng cao, Đại học bằng ti...",NaN
1,Ngành Kỹ thuật xây dựng công trình giao thông ...,"Thiết kế đường, thiết kế cầu, đồ án kết cấu bê...",- Thiết kế máy bay\n- Thiết kế tàu thủy\n- Thi...,bdebda50-cc98-422d-b5d8-3d77e662cf06,major,bdebda50-cc98-422d-b5d8-3d77e662cf06,"Thiết kế đường, thiết kế cầu, đồ án kết cấu bê...",C,"C. Thiết kế đường, thiết kế cầu, đồ án kết cấu...",C
2,Ngành Dược tại Trường Đại học Tôn Đức Thắng đà...,Toàn thời gian,- Bán thời gian\n- Từ xa\n- Liên kết đào tạo\n...,246435c7-1ad3-496c-8e2c-844fb093786b,major,index,Chính quy,C,C. Toàn thời gian\nD. Cấp tốc\nA. Liên kết đào...,C
3,Ngành Quản trị nguồn nhân lực liên quan đến nh...,"Quan hệ lao động, Bảo hộ lao động, Quản lý thể...",- Quản lý tài chính\n- Quản lý tiếp thị\n- Quả...,0b3d65ec-469c-41fb-9881-28616cf63b30,major,0b3d65ec-469c-41fb-9881-28616cf63b30,"Quan hệ lao động, Bảo hộ lao động, Quản lý thể...",A,"A. Quan hệ lao động, Bảo hộ lao động, Quản lý ...",A
4,Công nghệ sinh học là gì?,Công nghệ sinh học là ngành ứng dụng các kiến ...,- Công nghệ sinh học là ngành nghiên cứu về cá...,21456de7-cc54-4db5-86c3-001b38c6e16d,major,21456de7-cc54-4db5-86c3-001b38c6e16d,Công nghệ sinh học là ứng dụng các kiến thức s...,A,A. Công nghệ sinh học là ngành ứng dụng các ki...,NaN


In [2]:
df = df.fillna("None")
df.shape

(77, 10)

In [3]:
df.isna().sum()

Unnamed: 0       0
question         0
answer           0
wrong            0
doc_id           0
category         0
pred_doc_id      0
RAG_answer       0
correct_label    0
multi_prompt     0
dtype: int64

In [4]:
y = df['doc_id']
yhat = df['pred_doc_id']

score = 0
for a,b in zip(y, yhat):
    if a == b:
        score += 1

print(score/len(y))
print(score)
print(len(y))

0.5714285714285714
44
77


In [4]:
labels = df['RAG_label'].to_list()
preds = df['correct_label'].to_list()

correct = 0
for y, z in zip(labels, preds):
    if y == z:
        correct+=1
print(correct)
print(len(labels))
print(correct/len(labels))

from sklearn.metrics import classification_report

report = classification_report(y_true=labels, y_pred=preds)
print(report)

53
77
0.6883116883116883
              precision    recall  f1-score   support

           A       0.78      0.86      0.82        21
           B       0.75      0.92      0.83        13
           C       0.47      0.89      0.62         9
           D       0.71      0.94      0.81        16
         nan       0.00      0.00      0.00        18

    accuracy                           0.69        77
   macro avg       0.54      0.72      0.61        77
weighted avg       0.54      0.69      0.60        77



/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le